In [1]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [6]:
!ls drive/'My Drive'/'Colab Notebooks'/zttest

 RA_test.csv  'RA Test Instructions[8804].pdf'	 RA_test_Tian_Zhang.py


In [9]:
# -*- coding: utf-8 -*-
"""
Spyder Editor

This is a temporary script file.
"""

import pandas as pd
import numpy as np
import urllib2
import string
import calendar

def text_parser(doc_list):
    groupname_list = []
    submissiontype_list = []
    monthend_list = []
    prospectuses = []
    status = []
    
    for i in range(len(doc_list)):
        groupname_check = False
        submissiontype_check = False
        monthend_check = False
        temp_groupname = ""
        temp_submissiontype = ""
        temp_monthend = ""
        temp_status = "failed "
        
        # Retrive prospectuses of interest
        response = urllib2.urlopen(doc_list['link'].iloc[i])
        html_doc = response.read()
        
        # a. Extract header
        header = html_doc[html_doc.find('<SEC-HEADER>')+len('<SEC-HEADER>'):html_doc.find('</SEC-HEADER>')-1]
        header = header.split('\n')
    #     print(html_doc)
        # b. Extract following information from header
        # groupname
        # submission type
        # monthend
        
        for txt in header:
            if "COMPANY CONFORMED NAME:" in txt:
                temp_groupname = string.lower(txt[txt.rfind('\t')+1:].replace('\t',"").replace('\n',""))
            if "CONFORMED SUBMISSION TYPE:" in txt:
                temp_submissiontype = txt[txt.rfind('\t')+1:].replace('\t',"").replace('\n',"")
            if "FILED AS OF DATE:" in txt:
                temp_date = txt[txt.rfind('\t')+1:].replace('\t',"").replace('\n',"")
                temp_monthend = str(int(temp_date[4:6])) +'/' + str(calendar.monthrange(int(temp_date[:4]),int(temp_date[4:6]))[1]) +'/'+temp_date[:4]
            if temp_groupname != "" and temp_submissiontype != "" and temp_monthend !="":
                break
    #    print (temp_monthend, doc_list['monthend'].iloc[i])
        if temp_groupname == doc_list['groupname'].iloc[i]:
            groupname_check = True
        if temp_submissiontype == doc_list['submissiontype'].iloc[i]:
            submissiontype_check = True
        if temp_monthend == doc_list['monthend'].iloc[i]:
            monthend_check = True
        if groupname_check and submissiontype_check and monthend_check:
            groupname_list += [temp_groupname]
            submissiontype_list += [temp_submissiontype]
            monthend_list += [temp_monthend]
            status += ['processed '+temp_groupname]
            prospectuses += [html_doc[html_doc.find('<TEXT>')+len('<TEXT>'):html_doc.find('</TEXT>')-1]]
        else:
            temp_status += temp_groupname+"."
            if groupname_check is False:
                temp_status += ' groupname unmatched.'
            if submissiontype_check is False:
                temp_status += ' submissiontype unmatched.'
            if monthend_check is False:
                temp_status += ' monthend unmatched.'
            status += [temp_status]
    for line in status:
        print (line)

    df = pd.DataFrame({'groupname' : groupname_list, 'submissiontype':submissiontype_list, 'monthend':monthend_list, 'prospectus':prospectuses})
    return df   

def create_sql_table(df):
    #Create SQL table to install information
    from sqlalchemy import create_engine
    engine = create_engine('sqlite://', echo=False)
    df.to_sql('fund_prospectuses', con=engine)
    
    return engine



if __name__ == '__main__':
    #access to CSV
    doc_list = pd.read_csv("RA_test.csv")
    #print(doc_list)
    
    # Parse text from EDGAR links
    df = text_parser(doc_list)
    
    #Test the SQL table
    engine = create_sql_table(df)
    print (engine.execute("SELECT groupname, monthend FROM fund_prospectuses").fetchall())

ModuleNotFoundError: ignored

In [0]:
import pandas as pd

In [0]:
pd.read_csv('')